In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL.Image
import imageio
import tfutil
import matplotlib.pyplot as plt
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [2]:
network_path = "/om/user/shobhita/src/chexpert/CheXpert GAN/"
output_data_path = "/om/user/shobhita/src/chexpert/gan_fake_data/"
real_data_path = "/om/user/shobhita/src/chexpert/data/"

names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']

names_no_consolidation = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']

In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [5]:
# Initialize TensorFlow session.
tf.InteractiveSession()

# Import pretrained Chexpert GAN.
with open(network_path + "network-final.pkl", 'rb') as file:
    G, D, Gs = pickle.load(file)

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [29]:
labels = pd.read_csv(real_data_path + "CheXpert-v1.0-small/train_preprocessed.csv")
labels

,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,ID
0,173223,CheXpert-v1.0-small/train/patient40439/study1/...,Female,48,Frontal,AP,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40439
1,103369,CheXpert-v1.0-small/train/patient24820/study2/...,Female,69,Frontal,PA,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24820
2,55913,CheXpert-v1.0-small/train/patient13550/study3/...,Female,25,Lateral,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13550
3,192326,CheXpert-v1.0-small/train/patient46406/study1/...,Female,45,Frontal,AP,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46406
4,185864,CheXpert-v1.0-small/train/patient44121/study1/...,Male,51,Frontal,AP,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,44121
5,12776,CheXpert-v1.0-small/train/patient03153/study1/...,Female,63,Lateral,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3153
6,62816,CheXpert-v1.0-small/train/patient15161/study15...,Male,70,Frontal,PA,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,15161
7,138269,CheXpert-v1.0-small/train/patient33192/study5/...,Male,64,Lateral,NaN,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,33192
8,174779,CheXpert-v1.0-small/train/patient40875/study1/...,Female,73,Frontal,AP,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40875
9,16374,CheXpert-v1.0-small/train/patient04091/study6/...,Male,51,Frontal,AP,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4091


In [42]:
for name in names_no_consolidation:
    labels[name][labels[name] == -1] = 1
labels["Consolidation"][labels["Consolidation"] == -1] = 0
labels = labels.fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
len(labels)

201391

In [15]:
split = {}
total = len(labels)
for name in names:
    split[name] = sum(labels[name])/total


In [16]:
split

{'No Finding': 0.10015343287435884,
 'Enlarged Cardiomediastinum': 0.10352994920329112,
 'Cardiomegaly': 0.15786206930796312,
 'Lung Opacity': 0.4981950534035781,
 'Lung Lesion': 0.04787204989299423,
 'Edema': 0.29226728106022615,
 'Consolidation': 0.19005814559737028,
 'Pneumonia': 0.11100793977883817,
 'Atelectasis': 0.30024181815473383,
 'Pneumothorax': 0.10076418509268041,
 'Pleural Effusion': 0.43844064531185606,
 'Pleural Other': 0.02751860808079805,
 'Fracture': 0.04324920180147077,
 'Support Devices': 0.5248149122850574}

In [13]:
low = {name: value for name, value in split.items() if value/total <= 0.1}

In [17]:
lesion = low['Lung Lesion']
pleural = low['Pleural Other']
fracture = low['Fracture']

In [39]:
x, y, z = int(lesion*1.65), int(pleural*3.65), int(fracture*1.95)
new_total = x + y + z

print("Lesion: {}/{} + {} --> {}/{}".format(lesion, lesion/total, x, lesion+x, (lesion+x)/(total+new_total)))
print("Pleural: {}/{} + {} --> {}/{}".format(pleural, pleural/total, y, pleural+y, (pleural+y)/(total + new_total)))
print("Fracture: {}/{} + {} --> {}/{}".format(fracture, fracture/total, z, fracture+z, (fracture+z)/(total + new_total)))

Lesion: 9641.0/0.04787204989299423 + 15907 --> 25548.0/0.10038112451377156
Pleural: 5542.0/0.02751860808079805 + 20228 --> 25770.0/0.10125338886487761
Fracture: 8710.0/0.04324920180147077 + 16984 --> 25694.0/0.10095477584377824


In [53]:
label_vectors = {}
for cat in ["Lung Lesion", "Pleural Other", "Fracture"]:
    relevant_labels = labels[labels[cat] == 1]
    new_labels = relevant_labels.sample(x, replace=True)[names].to_numpy()
    label_vectors[cat] = new_labels

[[0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.]]
(15907, 14)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0.]]
(15907, 14)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.]]
(15907, 14)



In [47]:
labels = np.zeros((4, 14))
labels[0][4] = 1
labels[1][6] = 1
labels[2][11] = 1
labels[3][12] = 1

In [48]:
labels

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [ ]:
# Enlarged Cardiomediastinum, Lung Lesion, Consolidation, Pneumothorax

In [49]:
import time

batch = 1
used_labels = []
used_imgname = []
latents_raw = np.random.RandomState(1000).randn(labels.shape[0], *Gs.input_shapes[0][1:])
total_num = latents_raw.shape[0]

start_time = time.time()
seconds = 120


for n in range (int(total_num / batch)):
    current_time = time.time()
    if current_time - start_time > seconds:
        print("reached time limit")
        break
    
    latent_vec = latents_raw[n * batch: (n+1) * batch, :]
    label_vec = labels[n * batch: (n+1) * batch, :]
    used_labels.append(label_vec)
    images = Gs.run(latent_vec, label_vec)
    images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) # [-1,1] => [0,255]
    images = images.transpose(0, 2, 3, 1) # NCHW => NHWC
    save_images = np.squeeze(images, axis = -1)
    
    data_dir = output_data_path
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for idx in range(save_images.shape[0]):
        image_idx = idx + batch * n
        store_name = 'fake_%d.png' %image_idx
        used_imgname.append(store_name)
        store_path = os.path.join(data_dir, store_name)
        imageio.imwrite(store_path, save_images[idx])

print(len(input_labels))
print(len(used_labels))
print(len(used_imgname))

10000
4
4


## Replicate the training set (Example)

In [22]:
# Read the labels
labels = np.load('//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLearning/CheXpert_Dataset/labels/label_CheXpert.npy'
                , allow_pickle=True)
vals = labels.item().values()
values = np.array(list(vals))
labels = values
labels[:20]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

In [20]:
# Initialize TensorFlow session.
tf.InteractiveSession()

# Import official CelebA-HQ networks.
with open('network-snapshot-012000.pkl', 'rb') as file:
    G, D, Gs = pickle.load(file)

W1204 10:55:10.065437 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:468: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1204 10:55:10.067432 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:471: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1204 10:55:10.067432 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:471: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W1204 10:55:10.068429 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:472: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W1204 10:55:10.068429 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:475: The name tf.placeholder is deprecat

In [23]:
batch = 1
used_labels = []
used_imgname = []
latents_raw = np.random.RandomState(1000).randn(labels.shape[0], *Gs.input_shapes[0][1:])
total_num = latents_raw.shape[0]
for n in range (int(total_num / batch)):
    latent_vec = latents_raw[n * batch: (n+1) * batch, :]
    label_vec = labels[n * batch: (n+1) * batch, :]
    used_labels.append(label_vec)
    images = Gs.run(latent_vec, label_vec)
    images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) # [-1,1] => [0,255]
    images = images.transpose(0, 2, 3, 1) # NCHW => NHWC
    save_images = np.squeeze(images, axis = -1)
    
    data_dir = "//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLearning/Fake_Stanford/"
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for idx in range(save_images.shape[0]):
        image_idx = idx + batch * n
        store_name = 'fake_%d.png' %image_idx
        used_imgname.append(store_name)
        store_path = os.path.join(data_dir, store_name)
        imageio.imwrite(store_path, save_images[idx])

print(len(used_labels))
print(len(used_imgname))

191229
191229


In [25]:
labels[:20]
used_labels[:20]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

[array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.]]),
 array([[0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]]),
 array([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [30]:
tf.InteractiveSession()

# Import pretrained Chexpert GAN.
with open(network_path + "network-final.pkl", 'rb') as file:
    G, D, Gs = pickle.load(file)

real_labels = pd.read_csv(real_data_path + "CheXpert-v1.0-small/train_preprocessed.csv")

classes_to_generate = ["Lung Lesion", "Pleural Other", "Fracture"]
total = len(real_labels)

lesion =  sum(real_labels["Lung Lesion"])
pleural = sum(real_labels["Pleural Other"])
fracture = sum(real_labels["Fracture"])

lesion_n, pleural_n, fracture_n = int(lesion*1.65), int(pleural*3.65), int(fracture*1.95)
total_gen = total + lesion_n + pleural_n + fracture_n

print("Lesion: {}/{} + {} --> {}/{}".format(lesion, lesion/total, lesion_n, lesion+lesion_n, (lesion+lesion_n)/(total+total_gen)))
print("Pleural: {}/{} + {} --> {}/{}".format(pleural, pleural/total, pleural_n, pleural+pleural_n, (pleural+pleural_n)/(total + total_gen)))
print("Fracture: {}/{} + {} --> {}/{}".format(fracture, fracture/total, fracture_n, fracture+fracture_n, (fracture+fracture_n)/(total + total_gen)))
sys.stdout.flush()


label_vectors = {}
for cat, n in zip(classes_to_generate, [lesion_n, pleural_n, fracture_n]):
    relevant_labels = real_labels[real_labels[cat] == 1]
    new_labels = relevant_labels.sample(n, replace=True)[names].to_numpy()
    label_vectors[cat] = new_labels

for cat, arr in label_vectors.items():
    print("{}: {}".format(cat, arr.shape))



Lesion: 9641.0/0.04787204989299423 + 15907 --> 25548.0/0.0560384820388637
Pleural: 5542.0/0.02751860808079805 + 20228 --> 25770.0/0.056525429863062374
Fracture: 8710.0/0.04324920180147077 + 16984 --> 25694.0/0.056358727004327695
Lung Lesion: (15907, 14)
Pleural Other: (20228, 14)
Fracture: (16984, 14)


In [36]:
labels_save = {}
for cat in classes_to_generate:
    labels = label_vectors[cat]
    batch = 1
    used_labels = []
    used_imgname = []
    latents_raw = np.random.RandomState(1000).randn(labels.shape[0], *Gs.input_shapes[0][1:])
    total_num = latents_raw.shape[0]

    print("Generating {}".format(cat))
    sys.stdout.flush()

    for n in range(int(total_num / batch)):
        if n % 100 == 0:
            print("{}/{}".format(n, total_num))

        latent_vec = latents_raw[n * batch: (n + 1) * batch, :]
        label_vec = labels[n * batch: (n + 1) * batch, :]
        used_labels.append(label_vec)
        images = Gs.run(latent_vec, label_vec)
        images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8)  # [-1,1] => [0,255]
        images = images.transpose(0, 2, 3, 1)  # NCHW => NHWC
        save_images = np.squeeze(images, axis=-1)

        data_dir = output_data_path
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)

        for idx in range(save_images.shape[0]):
            image_idx = idx + batch * n
            labels_save["{}_{}".format(cat, image_idx)] = labels[image_idx, :]
            store_name = 'fake_{}_{}.png'.format(cat, image_idx)
            used_imgname.append(store_name)
            store_path = os.path.join(data_dir, store_name)
            imageio.imwrite(store_path, save_images[idx])
    
    print("Done with {}".format(cat))
    print(len(labels))
    print(len(used_labels))
    print(len(used_imgname))
    sys.stdout.flush()

with open(output_data_path + "gan_image_labels.pkl", "wb") as handle:
    pickle.dump(labels_save, handle)
sys.stdout.flush()

Generating Lung Lesion
0/15907
100/15907
200/15907
300/15907
400/15907
500/15907
600/15907
700/15907
800/15907
900/15907
1000/15907
1100/15907
1200/15907
1300/15907
1400/15907
1500/15907
1600/15907
1700/15907
1800/15907
1900/15907
2000/15907
2100/15907
2200/15907
2300/15907
2400/15907
2500/15907
2600/15907
2700/15907
2800/15907
2900/15907
3000/15907
3100/15907
3200/15907
3300/15907
3400/15907
3500/15907
3600/15907
3700/15907
3800/15907
3900/15907
4000/15907
4100/15907
4200/15907
4300/15907
4400/15907
4500/15907
4600/15907
4700/15907
4800/15907
4900/15907
5000/15907
5100/15907
5200/15907
5300/15907
5400/15907
5500/15907
5600/15907
5700/15907
5800/15907
5900/15907
6000/15907
6100/15907
6200/15907
6300/15907
6400/15907
6500/15907
6600/15907
6700/15907
6800/15907
6900/15907
7000/15907
7100/15907
7200/15907
7300/15907
7400/15907
7500/15907
7600/15907
7700/15907
7800/15907
7900/15907
8000/15907
8100/15907
8200/15907
8300/15907
8400/15907
8500/15907
8600/15907
8700/15907
8800/15907
8900/15907

In [24]:
with open(output_data_path + "gan_image_labels.pkl", "rb") as handle:
    l = pickle.load(handle)
l

{'Lung Lesion_0': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Pleural Other_0': array([0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.]),
 'Fracture_0': array([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.])}